# Symmetry reduction

**Adapted from**: [SymbolicWedderburn example](https://github.com/kalmarek/SymbolicWedderburn.jl/blob/tw/ex_sos/examples/ex_C4.jl)

In [1]:
import MutableArithmetics as MA
using MultivariatePolynomials
using MultivariateBases

using DynamicPolynomials
@polyvar x[1:4]

(DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[x₁, x₂, x₃, x₄],)

We would like to find the minimum value of the polynomial

In [2]:
poly = sum(x) + sum(x.^2)

x₄ + x₃ + x₂ + x₁ + x₄² + x₃² + x₂² + x₁²

As we can decouple the problem for each `x[i]` for which `x[i] + x[i]^2` has
minimum value 0.25, we would expect to get `-1` as answer.
Can this decoupling be exploited by SumOfSquares as well ?
For this, we need to use a certificate that can exploit the permutation symmetry of the polynomial.

In [3]:
using SumOfSquares

We define the symmetry group as a permutation group in the variables.
In order to do that, we define the action of a permutation on a monomial
as the monomial obtained after permuting the variables.

In [4]:
using PermutationGroups
G = PermGroup([perm"(1,2,3,4)"])

Permutation group on 1 generator generated by
 (1,2,3,4)

We can use this certificate as follows:

In [5]:
import CSDP
solver = CSDP.Optimizer
model = Model(solver)
@variable(model, t)
@objective(model, Max, t)
pattern = Symmetry.Pattern(G, Symmetry.VariablePermutation())
con_ref = @constraint(model, poly - t in SOSCone(), symmetry = pattern)
optimize!(model)
value(t)

Iter: 10 Ap: 9.00e-01 Pobj: -1.0000000e+00 Ad: 1.00e+00 Dobj: -1.0000000e+00 
Success: SDP solved
Primal objective value: -1.0000000e+00 
Dual objective value: -1.0000000e+00 
Relative primal infeasibility: 4.04e-14 
Relative dual infeasibility: 2.78e-10 
Real Relative Gap: 2.16e-09 
XZ Relative Gap: 2.71e-09 
DIMACS error measures: 4.35e-14 0.00e+00 5.75e-10 0.00e+00 2.16e-09 2.71e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -6.9333327e+00 Ad: 9.27e-01 Dobj:  2.5022292e+01 
Iter:  2 Ap: 1.00e+00 Pobj: -3.3371292e+00 Ad: 9.16e-01 Dobj:  4.2284146e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -1.1686719e+00 Ad: 8.76e-01 Dobj:  1.7691868e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0108916e+00 Ad: 9.30e-01 Dobj: -8.8720183e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.0008407e+00 Ad: 9.56e-01 Dobj: -9.9267833e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.0000536e+00 Ad: 9.78e-01 Dobj: -9.9968404e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -1.0000039e+00 Ad: 1.00e+0

-1.0000000005215735

We indeed find `-1`, let's verify that symmetry was exploited:

In [6]:
gram_matrix(con_ref).blocks

4-element Vector{GramMatrix{Float64, FixedPolynomialBasis{DynamicPolynomials.Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}, Float64}, Vector{DynamicPolynomials.Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}, Float64}}}, Float64, SymMatrix{Float64}}}:
 GramMatrix with row/column basis:
 FixedPolynomialBasis([1.0, -0.5*x[4] - 0.5*x[3] - 0.5*x[2] - 0.5*x[1]])
And entries in a 2×2 SymMatrix{Float64}:
  1.0000000005215735  -1.0000000000000018
 -1.0000000000000018   0.9999999999999988
 GramMatrix with row/column basis:
 FixedPolynomialBasis([-0.7071067811865472*x[4] + 0.7071067811865475*x[2]])
And entries in a 1×1 SymMatrix{Float64}:
 0.9999999999999998
 GramMatrix with row/column basis:
 FixedPolynomialBasis([-0.7071067811865472*x[3] + 0.7071067811865475*x[1]])
And entries in a 1×1 SymMatrix{Float64}:
 

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*